# 영화 리뷰 크롤링

## 1. 환경 구축

- Selenium
- BeautifulSoup
- pandas

In [12]:
# %pip install Selenium
# %pip install beautifulsoup4
# %pip install pandas

In [13]:
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import time
import pandas as pd
from selenium.common.exceptions import NoSuchElementException

In [14]:
cgv_movie_codes = {"moana2":"88381", "wicked":"88076", "1승":"89075", "gladiator2":"88459"}
CGV_URL = "http://www.cgv.co.kr/movies/detail-view/?midx="
CGV_OBJECT_NAME = "cgv_reviews.csv"
CGV_DATA_PATH = f"./data/{CGV_OBJECT_NAME}"

megabox_movie_codes = {"moana2":"24036800", "wicked":"24010200", "1승":"24073500", "gladiator2":"24043900"}
MEGABOX_URL = "https://www.megabox.co.kr/movie-detail/comment?rpstMovieNo="
MEGABOX_OBJECT_NAME = "megabox_reviews.csv"
MEGABOX_DATA_PATH = f"./data/{MEGABOX_OBJECT_NAME}"

# LOTTECINEMA_URL = "https://www.lottecinema.co.kr/NLCHS/Movie/MovieDetailView?movie=20881"
# LOTTECINEMA_DATA_PATH = "./data/lottecinema_reviews.csv"

In [15]:
def get_movie_reviews_on_cgv(url, page_num=10):
    """
    CGV는 한페이지당 리뷰 6개를 가지고 온다.
    """
    wd = webdriver.Chrome()
    wd.get(url)
    # 빈 리스트 생성하기
    review_list=[]
    
    for page_no in range(1,page_num+1): # 1페이지에서 page_num까지의 리뷰 추출
        try:
            page_ul = wd.find_element(By.ID, 'paging_point') # 페이지 포인트 코드 추출
            page_a = page_ul.find_element(By.LINK_TEXT, str(page_no))
            page_a.click()
            time.sleep(2) # 페이지 로딩까지의 시간 두기

            reviews = wd.find_elements(By.CLASS_NAME, 'box-comment')
            review_list += [ review.text for review in reviews ]

            if page_no % 10 == 0: # 10이상의 값을 만났을 때
                next_button = page_ul.find_element(By.XPATH, './/button[contains(@class, "btn-paging next")]')
                next_button.click()
                time.sleep(2)
        except NoSuchElementException as e:
                print("불러올 페이지가 없습니다.")
                print(e)
                break
    movie_review_df = pd.DataFrame({"review" : review_list})
    wd.close()
    return movie_review_df

In [16]:
full_url = CGV_URL + cgv_movie_codes["moana2"]

review_df = get_movie_reviews_on_cgv(url=full_url, page_num=5)

review_df.to_csv(CGV_DATA_PATH, sep=",",index=False, encoding="utf-8")

In [17]:
def get_movie_reviews_on_megabox(url, page_num=3):
    """
    메가박스는 한페이지당 리뷰 10개를 가지고 온다.
    """
    wd = webdriver.Chrome()
    wd.get(url)
    # 빈 리스트 생성하기
    review_list=[]
    
    for page_no in range(1,page_num+1): # 1페이지에서 page_num까지의 리뷰 추출
        try:
            if page_no % 10 != 1:
                page_nav = wd.find_element(By.CLASS_NAME, 'pagination') # 페이지 포인트 코드 추출
                page_a = page_nav.find_element(By.LINK_TEXT, str(page_no))
                page_a.click()
                time.sleep(2) # 페이지 로딩까지의 시간 두기

            reviews = wd.find_elements(By.CLASS_NAME, 'story-txt')
            review_list += [ review.text for review in reviews ]

            if page_no % 10 == 0: # 10이상의 값을 만났을 때
                next_button = page_nav.find_element(By.XPATH, './/a[contains(@class, "control next")]')
                next_button.click()
                time.sleep(2)
        except NoSuchElementException as e:
            print("불러올 페이지가 없습니다.")
            print(e)
            break
    movie_review_df = pd.DataFrame({"review" : review_list})
    wd.close()
    return movie_review_df

In [ ]:
full_url = MEGABOX_URL + megabox_movie_codes["moana2"]

review_df = get_movie_reviews_on_megabox(url=full_url, page_num=3)

review_df.to_csv(MEGABOX_DATA_PATH, sep=",",index=False, encoding="utf-8")

In [19]:
# def get_movie_reviews_on_lottecinema(url, page_num=-1):
#     wd = webdriver.Chrome()
#     wd.get(url)
#     # 빈 리스트 생성하기
#     review_list=[]
#     i = 1

#     time.sleep(2)
#     element = wd.find_element(By.XPATH, '//*[@id="contents_new23"]/div[2]/ul/li[2]/button')
#     wd.execute_script("arguments[0].scrollIntoView(true);", element)  # 스크롤로 요소를 화면에 보이도록 이동
#     ActionChains(wd).move_to_element(element).perform()  # 요소 위로 마우스 이동
#     time.sleep(1) # 페이지 로딩까지의 시간 두기
#     # wd.execute_script("arguments[0].click();", tab_btn)
#     # WebDriverWait(wd, 3).until(
#     #     EC.element_to_be_clickable((By.XPATH, '//*[@id="contents_new23"]/div[2]/ul/li[2]/button'))
#     # ).click()
#     # time.sleep(1) # 페이지 로딩까지의 시간 두기
    
#     # while True if page_num == -1 else i <= page_num:
#     #     try:
#     #         next_btn = wd.find_element(By.ID, 'btn_review_more')
#     #         if next_btn.is_displayed() and i != page_num:
#     #             wd.execute_script("arguments[0].scrollIntoView(true);", next_btn)  # 스크롤로 요소를 화면에 보이도록 이동
#     #             ActionChains(wd).move_to_element(next_btn).perform()  # 요소 위로 마우스 이동
#     #             time.sleep(2)
#     #             WebDriverWait(wd, 10).until(
#     #                 EC.element_to_be_clickable((By.ID, 'btn_review_more'))
#     #             ).click()
#     #             i += 1
#     #             time.sleep(2) # 페이지 로딩까지의 시간 두기
#     #         else:
#     #             reviews = wd.find_elements(By.CLASS_NAME, 'review_info')
#     #             review_list += [ review.text for review in reviews ]
#     #             break
#     #     except NoSuchElementException as e:
#     #         print("불러올 페이지가 없습니다.")
#     #         print(e)
#     #         reviews = wd.find_elements(By.CLASS_NAME, 'review_info')
#     #         review_list += [ review.text for review in reviews ]
#     #         break
        
#     movie_review_df = pd.DataFrame({"review" : review_list})
#     # wd.close()
#     return movie_review_df

In [20]:
# review_df = get_movie_reviews_on_lottecinema(url=LOTTECINEMA_URL, page_num=10)

# review_df.to_csv(LOTTECINEMA_DATA_PATH, sep=",",index=False, encoding="utf-8")